In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import multiprocessing
import subprocess
import pickle
import json
import time

from model_template import MODEL_TEMPLATE

In [2]:
DIR_EXPERIMENTS = "../experiments"
DIR_CODEX = "codex"
DIR_MODELS = "models"
DIR_RESULTS = "results"
DIR_WEBPPL = "webppl"

EXPERIMENT_ID = "run-2023-01-27-11-52-34"
FILE_PROMPTING_RESULTS = os.path.join(DIR_EXPERIMENTS, EXPERIMENT_ID, DIR_CODEX, "results.csv")

In [3]:
df = pd.read_csv(FILE_PROMPTING_RESULTS, index_col="task_id", keep_default_na=False)
df

,language_full,language_phrase_1,language_phrase_2,language_phrase_3,code_phrase_1,code_phrase_2,code_phrase_3,choice_index,finish_reason,text,codex_language_phrase_1,codex_code_phrase_1,codex_language_phrase_2,codex_code_phrase_2,codex_language_phrase_3,codex_code_phrase_3
task_id,,,,,,,,,,,,,,,,
1,There is a stack of yellow blocks on the left ...,There is a stack of yellow blocks on the left ...,There are a few red blocks on the middle of th...,,"condition(filter(isYellow, filter(isOnLeft, wo...","condition(filter(isOnMiddle, filter(isRed, wor...",,0,stop,\n// There is a stack of yellow blocks on the ...,There is a stack of yellow blocks on the left ...,"condition(filter(isOnLeft, (filter(isYellow, w...",There are a few red blocks on the middle of th...,"condition(filter(isOnMiddle, (filter(isRed, wo...",,
2,There is a tall stack of yellow blocks on the ...,There is a tall stack of yellow blocks on the ...,There are some red blocks near the yellow stack.,,"condition(filter(isTall, filter(isOnMiddle, fi...","condition(filter(isRed, filter(isNear(filter(i...",,0,stop,\n// There is a tall stack of yellow blocks on...,There is a tall stack of yellow blocks on the ...,"condition(filter(isTall, filter(isOnMiddle, (f...",There are some red blocks near the yellow stack.,"condition(filter(isNear(filter(isTall, filter(...",,
3,"Half of the blocks are red, and half of the bl...",Half of the blocks are red.,Half of the blocks are yellow.,,"condition(filter(isRed, world.blocks).length =...","condition(filter(isYellow, world.blocks).lengt...",,0,stop,\n// Half of the blocks are red.\ncondition(fi...,Half of the blocks are red.,"condition(filter(isRed, world.blocks).length =...",Half of the blocks are yellow.,"condition(filter(isYellow, world.blocks).lengt...",,
4,"There are several stacks of yellow blocks, and...",There are several stacks of yellow blocks.,There is one stack of red blocks.,,"condition(filter(isYellow, world.stacks).lengt...","condition(filter(isRed, world.stacks).length =...",,0,stop,\n// There are several stacks of yellow blocks...,There are several stacks of yellow blocks.,"condition(filter(isYellow, world.stacks).lengt...",There is one stack of red blocks.,"condition(filter(isRed, world.stacks).length =...",,
5,There are two stacks of yellow blocks on the l...,There are two stacks of yellow blocks on the l...,There are also two stacks of red blocks on the...,,"condition(filter(isOnLeft, (filter(isYellow, w...","condition(filter(isOnRight, (filter(isRed, wor...",,0,stop,\n// There are two stacks of yellow blocks on ...,There are two stacks of yellow blocks on the l...,"condition(filter(isOnLeft, (filter(isYellow, w...",There are also two stacks of red blocks on the...,"condition(filter(isOnRight, (filter(isRed, wor...",,
6,There are two stacks of yellow blocks on the l...,There are two stacks of yellow blocks on the l...,There are also two stacks of red blocks on the...,The table is bumped from the left.,"condition(filter(isOnLeft, (filter(isYellow, w...","condition(filter(isOnRight, (filter(isRed, wor...",condition(isOnLeft(world.force));,0,stop,\n// There are two stacks of yellow blocks on ...,There are two stacks of yellow blocks on the l...,"condition(filter(isOnLeft, (filter(isYellow, w...",There are also two stacks of red blocks on the...,"condition(filter(isOnRight, (filter(isRed, wor...",The table is bumped from the left.,condition(isOnLeft(world.force));
7,"There is a short stack of red blocks, and ther...",There is a short stack of red blocks.,There is a tall stack of yellow blocks near th...,,"condition(filter(isShort, filter(isRed, world....","condition(filter(isTall, filter(isYellow, filt...",,0,stop,\n// There is a short stack of red blocks.\nco...,There is a short stack of red blocks.,"condition(filter(isShort, filter(isRed, world....",There is a tall stack of yellow blocks near th...,"condition(filter(isTall, filter(isYellow, filt...",,
8,There are many stacks of blocks on the table. ...,There are many s

In [4]:
def populate_template(df, task_id, samples: int = 10):
    conditions = []
    for i in range(1, 4):
        l = df.loc[task_id, f"codex_language_phrase_{i}"]
        c = df.loc[task_id, f"codex_code_phrase_{i}"]
        if c:
            conditions += ["// " + l, c + "\n"]
    conditions_str = "\n    ".join(conditions)
    
    model_template = MODEL_TEMPLATE.format(SAMPLES=samples, CONDITIONS=conditions_str)
    return model_template

In [5]:
print(f"{populate_template(df, 1)}")

var makeBlockWorld = function () {

    //// Distributions and parameters ////

    var truncGeom = function (p, m, n) {
      if (m > n) {
        return uniformDraw(_.range(1, n + 1));
      } else {
        return flip(p) ? truncGeom(p, m + 1, n) : m;
      }
    }

    var dim = 10;
    var tableSize = 100;
    var worldWidth = 600;
    var worldHeight = 500;
    var color = function () { return flip() ? 'red' : 'yellow' };
    var monoColor = flip();
    var stackHeight = function () { return truncGeom(0.5, 1, 8) };
    var numStacks = truncGeom(0.5, 1, 8);
    var xpositions = _.range(worldWidth / 2 - tableSize, worldWidth / 2 + tableSize + 20, 20);

    //// Object definitions ////

    var ground = {
      shape: 'rect',
      static: true,
      dims: [100000 * worldWidth, 10],
      x: worldWidth / 2,
      y: worldHeight
    }

    var table = {
      shape: 'rect',
      static: false,
      dims: [tableSize, tableSize],
      x: worldWidth / 2,
      y: 390,
      color: '

In [ ]:
def write_model_for_task(df, task_id, samples, model_dir):
    model_template = populate_template(df, task_id, samples=samples)
    model_file = os.path.join(model_dir, f"model_{task_id:03d}.wppl")
    with open(model_file, "w") as f:
        f.write(model_template)

def run_single_task(task_id, model_dir, results_dir, use_cached: bool = False, timeout: int = 120, seed: int = 123):
    model_file = os.path.join(model_dir, f"model_{task_id:03d}.wppl")
    results_file = os.path.join(results_dir, f"results_{task_id:03d}.pkl")

    # Optionally, load cached prior result
    if use_cached and os.path.exists(results_file):
        with open(results_file, "rb") as f:
            print(f"Loaded cached {results_file}")
            return pickle.load(f)
    
    cmd = f"webppl {model_file} --require node_modules/physics --random-seed {seed}"
    print(f"Running task {task_id}:\n{cmd}")

    time_start = time.time()
    try:
        completed_process = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
    except:
        print(f"WARNING: Exceeded {timeout}s timeout while running task {task_id}")
        completed_process = None

    if completed_process is not None:
        setattr(completed_process, "runtime", time.time() - time_start)
        print(f"Simulation completed in {completed_process.runtime:.3f}s")
        if completed_process.returncode == 0:
            print(completed_process.stdout)
        else:
            print(f"WARNING: WebPPL encountered error running task {task_id}:")
            print(completed_process.stderr)

    with open(results_file, "wb") as f:
        pickle.dump(completed_process, f)
        
    return completed_process

def run_experiment(experiment_id: str, use_cached: bool = False, samples: int = 10, timeout: int = 120, seed: int = 123):
    model_dir = os.path.join(DIR_EXPERIMENTS, experiment_id, DIR_WEBPPL, DIR_MODELS)
    results_dir = os.path.join(DIR_EXPERIMENTS, experiment_id, DIR_WEBPPL, DIR_RESULTS)
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(results_dir, exist_ok=True)
    
    df = pd.read_csv(os.path.join(DIR_EXPERIMENTS, experiment_id, DIR_CODEX, "results.csv"), index_col="task_id", keep_default_na=False)
    
    # Write out model files
    for task_id, row in df.iterrows():
        write_model_for_task(df, task_id, samples, model_dir)
        
    # Run simulations
    pool = multiprocessing.Pool(os.cpu_count())
    print(f"Initialized multiprocessing pool with {os.cpu_count()} workers.")
    
    args_list = [(task_id, model_dir, results_dir) for task_id in df.index]
    kwargs = {
        "use_cached": use_cached,
        "timeout": timeout,
        "seed": seed,
    }

    job_list = [pool.apply_async(run_single_task, args, kwargs) for args in args_list]
    completed_process_list = [job.get() for job in job_list]

    # Terminate multiprocessing
    pool.close()
    pool.join()
    pool.terminate()
    
    # completed_process_list = []
    # for task_id, row in df.iterrows():
    #     completed_process = run_single_task(task_id, model_dir, results_dir, use_cached=use_cached, timeout=timeout, seed=seed)
    #     completed_process_list.append(completed_process)
        
    # Parse simulation results
    results_list = []
    for i, completed_process in enumerate(completed_process_list):
        task_id = i + 1

        if completed_process is not None:
            if completed_process.returncode == 0:
                results_json = json.loads(completed_process.stdout)
                results_list.append({
                    "task_id": task_id,
                    "probs": results_json["probs"],
                    "runtime": completed_process.runtime,
                    "support": results_json["support"],
                    "stderr": None,
                })
            else:
                print(f"WARNING: No results for task {task_id}")
                results_list.append({
                    "task_id": task_id,
                    "runtime": completed_process.runtime,
                    "stderr": completed_process.stderr,
                })
        else:
            print(f"WARNING: No results for task {task_id}")
            results_list.append({
                "task_id": task_id,
            })

    df_results = pd.DataFrame(results_list)
    df_results = df_results.set_index("task_id")
    df_results.to_csv(os.path.join(DIR_EXPERIMENTS, experiment_id, DIR_WEBPPL, "simulator_results.csv"))
    
    return df_results

df_results = run_experiment(experiment_id=EXPERIMENT_ID, use_cached=True, samples=1, timeout=10)

Initialized multiprocessing pool with 10 workers.


Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/physlang/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/physlang/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/physlang/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/physlang/lib/python3.9/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'run_single_task' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/physlang/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/physlang/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/ana

In [ ]:
df_results